Python OpenCV: Optical Flow with Lucas-Kanade method

In [ ]:
import numpy as np
import cv2

# Load the video
cap = cv2.VideoCapture("./Assets/car3.mp4")
  

# Get the first frame
ret, frame1 = cap.read()

# Convert to grayscale
prev_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

# Create a mask for drawing the optical flow vectors
mask = np.zeros_like(frame1)

# Set the Lucas-Kanade parameters
lk_params = dict(winSize=(15, 15),maxLevel=4,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Define the points to track
# You can either select them manually or use an algorithm like Shi-Tomasi corner detection
# Here, we'll use Shi-Tomasi corner detection to automatically detect the points
feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)

# Get the points to track
p0 = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)

# Loop over the frames
while True:
    # Get the next frame
    ret, frame2 = cap.read()

    # Convert to grayscale
    next_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

    # Calculate the optical flow using the Lucas-Kanade method
    p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, next_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st == 1]
    good_old = p0[st == 1]

    # Draw the optical flow vectors
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
        frame2 = cv2.circle(frame2, (int(a), int(b)), 5, (0, 0, 255), -1)

    img = cv2.add(frame2, mask)

    # Display the result
    cv2.imshow('frame', img)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Update the previous frame and points
    prev_gray = next_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

# Release the video capture and destroy the windows
cap.release()
cv2.destroyAllWindows()
